In [1]:
from pathlib import Path
import json
import sys
import os

import gtfstk as gt
import pandas as pd
import numpy as np
import shapely.geometry as sg
import folium

sys.path.append('../')

import gtfs_map_matcher as mm

%load_ext autoreload
%autoreload 2



# Load sample GTFS feed

In [3]:
path = mm.DATA_DIR/'auckland_gtfs_sample.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,"[Go West, AT Metro, Fullers Ferries, Pavlovich..."
1,timezone,Pacific/Auckland
2,start_date,20171012
3,end_date,20171220
4,num_routes,15
5,num_trips,4675
6,num_stops,1022
7,num_shapes,195
8,sample_date,20171019
9,num_routes_active_on_sample_date,15


# Create sample points

In [11]:
t = feed.trips.merge(feed.routes)
cond = t['route_type'] == 3
t = t[cond].copy()

%time spoints_1 = mm.sample_trip_points(feed, t.trip_id, 5)
%time spoints_2 = mm.sample_trip_points(feed, t.trip_id, 100)
%time spoints_3 = mm.sample_trip_points(feed, t.trip_id, point_dist=0.1)

print('num stop patterns = ', len(spoints_2))
pattern = list(spoints_2.keys())[0]


CPU times: user 520 ms, sys: 4 ms, total: 524 ms
Wall time: 518 ms
CPU times: user 708 ms, sys: 4 ms, total: 712 ms
Wall time: 704 ms
CPU times: user 892 ms, sys: 4 ms, total: 896 ms
Wall time: 893 ms
num stop patterns =  50


# Compare sample points with one actual feed shape

In [12]:
l1 = sg.LineString(spoints_1[pattern])
l2 = sg.LineString(spoints_2[pattern])
l3 = sg.LineString(spoints_3[pattern])

# Get actual trip shape
p = mm.get_stop_patterns(feed)
shid = p.loc[p['stop_pattern'] == pattern, 'shape_id'].unique()[0]
la = feed.build_geometry_by_shape(shape_ids=[shid], use_utm=False)[shid]

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(l1), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
folium.GeoJson(sg.mapping(l2), 
  style_function=lambda x: {'color': 'orange'}).add_to(mappy)
folium.GeoJson(sg.mapping(l3), 
  style_function=lambda x: {'color': 'yellow'}).add_to(mappy)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
mappy

# Match one set of sample points using various matching services and compare to GTFS shape

In [13]:
# Map match with Mapzen

mpoints = mm.match_with_mapzen({pattern: spoints_2[pattern]},
  os.environ['MAPZEN_KEY'])[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


KeyError: '7345-20171003151834_v59.7-7562-20171003151834_v59.7-7399-20171003151834_v59.7-7397-20171003151834_v59.7-7242-20171003151834_v59.7-8860-20171003151834_v59.7-7383-20171003151834_v59.7-7965-20171003151834_v59.7-7963-20171003151834_v59.7-7961-20171003151834_v59.7-7127-20171003151834_v59.7-7257-20171003151834_v59.7-7833-20171003151834_v59.7-7831-20171003151834_v59.7-7280-20171003151834_v59.7-7821-20171003151834_v59.7-7455-20171003151834_v59.7-7453-20171003151834_v59.7-7451-20171003151834_v59.7-7449-20171003151834_v59.7-7447-20171003151834_v59.7-7445-20171003151834_v59.7-7443-20171003151834_v59.7-7441-20171003151834_v59.7-7439-20171003151834_v59.7-7437-20171003151834_v59.7-7435-20171003151834_v59.7-7433-20171003151834_v59.7-7431-20171003151834_v59.7-7429-20171003151834_v59.7-7427-20171003151834_v59.7-7425-20171003151834_v59.7-7423-20171003151834_v59.7-7421-20171003151834_v59.7-8073-20171003151834_v59.7-8071-20171003151834_v59.7-1718-20171003151834_v59.7-8069-20171003151834_v59.7-8

In [ ]:
# Map match with OSRM

mpoints = mm.match_with_osrm({pattern: spoints_2[pattern]})[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with Mapbox, which uses OSRM

mpoints = mm.match_with_mapbox({pattern: spoints_2[pattern]},
  os.environ['MAPBOX_KEY'])[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with Google

mpoints = mm.match_with_google({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Test batch map match

n = 30
print('num requests =', n)
test = {k: v for k, v in list(spoints_2.items())[:n]}
#api_key = mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY')
api_key = os.environ['MAPZEN_KEY']

%time mpoints = mm.map_match(test, 'mapzen', api_key)

In [ ]:
print(mm.get_num_match_calls(feed))

In [ ]:
t = feed.trips.merge(feed.routes)
trip_id, shape_id = t.loc[t['route_type'] == 3, ['trip_id', 'shape_id']].iloc[0].values
print(trip_id, shape_id)
trip_ids = [trip_id]
%time feed2 = mm.create_shapes(feed, 'google', mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'), trip_ids=trip_ids)

In [ ]:
print(feed.shapes[feed.shapes.shape_id == shape_id])
print(feed2.shapes[feed2.shapes.shape_id == shape_id])